https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/metadata_extraction/EntityExtractionClimate.ipynb

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
from llama_index.llms.openai import OpenAI


os.environ["OPENAI_API_KEY"] = ""
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

In [3]:
import re
from llama_index.core.schema import TransformComponent


class TextCleaner(TransformComponent):
    def __call__(self, nodes, **kwargs):
        for node in nodes:
            node.text = re.sub(r"[^0-9A-Za-z ]", "", node.text)
        return nodes

In [4]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import (
    QuestionsAnsweredExtractor,
)
from llama_index.core.node_parser import SentenceSplitter

pipeline = IngestionPipeline(
    transformations=[
        TextCleaner(),
        SentenceSplitter(chunk_size=512),
        #TokenTextSplitter(chunk_size=512),
        #QuestionsAnsweredExtractor(llm=llm, questions=3)
    ],
)

In [5]:
from llama_index.core.schema import MetadataMode
from llama_index.core import SimpleDirectoryReader
from rich import print as rprint

documents = SimpleDirectoryReader("./data/").load_data()

In [6]:
nodes = pipeline.run(documents=documents,
                    in_place=True,
                    show_progress=True,)
i = 0

c:\Users\n.fretti\Desktop\projects\rag_and_roll\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 81/81 [00:00<00:00, 165.76it/s]


In [7]:
rprint(nodes[i].get_content(
    metadata_mode=MetadataMode.LLM
))
i+=1


page_label: 1
file_path: c:\Users\n.fretti\Desktop\projects\rag_and_roll\data\billionaires_page.pdf

7123 1 131 PM The Worlds Billionaires  Wikipediahttpsenwikipediaor gwikiTheW orld27sBillionaires 133The Worlds 
BillionairesList of the worlds billionaires ranked in order of net worthThe net worth of the worlds billionaires 
increased fromless than US1 trillion in 2000 to over 7 trillion in 2015Publication detailsPublisher Whale Media 
InvestmentsForbes familyPublication F o r b e sFirst published March 1987 1 Latest publication April 4 2023Current 
list details 2023 2 Wealthiest Bernard ArnaultNet worth 1st US211billionNumber ofbillionaires2640 from 2668Total 
list net worthvalueUS122trillion from US 127trillionNumber of women 337New members to thelist150F o r b e s The 
Worlds Billionaires website httpswwwforbescombillionairesThe Worlds BillionairesThe Worlds Billionaires  is an 
annualranking of people who are considered to have anet worth  of 1 billion or more by theAmerican busines s 
magazine Forbes  The listwas first published in March 1987 1 The totalnet worth of each individual on the list 
isestimated and is cited in United States dollars based on their documented assets andaccounting for debt and other
factors Royaltyand dictators  whose wealth comes from theirpositions are excluded from these lists 3 Thisranking is
an index of the wealthiestdocumented individuals excluding any rankingof those with wealth that is not able to 
becompletely ascertained 4 In 2018  Amazon  founder  Jeff Bezos wasranked at the top for the first time and 
becamethe first centibillionaire included in theranking 5  surpassing Microsoft  founder BillGates  who had topped 
the list 18 of theprevious 24 years In 2022 after topping thelist for four years Bezos was surpassed by ElonMusk  6
In 2023  Musk was in turn surpass edby French businessman Bernard Arnault  aftertopping the list for just a year 
Arnault becamethe first French person to top the list 2 Each year Forbes  employs a team of over 50reporters from a
variety of countri es to trackthe activity of the worlds wealthiestindividuals 7  and sometimes groups or families 
who share wealth Preliminary surveys

In [8]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(nodes=nodes)
engine = index.as_query_engine()

In [ ]:
rprint(engine.query("Who is the author of the book?"))

Paul Graham


In [ ]:
rprint(engine.query("What inspired the author to switch from studying philosophy to studying AI in college?"))

The author was inspired to switch from studying philosophy to studying AI in college after being drawn into the world of AI through a novel by Heinlein called "The Moon is a Harsh Mistress" and a PBS documentary showing Terry Winograd using SHRDLU.


In [11]:
rprint(engine.query("What would the author say about art vs. engineering?").response)

The author expresses a shift in interest from engineering to art, highlighting the appeal of creating lasting works
like paintings compared to the impermanence of software development.

In [12]:
rprint(engine.query("Why did the author have to learn italian?").response)

The author had to learn Italian because foreigners (stranieri) were required to take an entrance exam to study art 
at the Accademia di Belli Arti in Florence.

In [13]:
rprint(engine.query("Why the author was in Florence?").response)

The author was in Florence to take the entrance exam at the Accademia di Belli Arti, which was the oldest art 
school and where the author imagined they would receive a good education in art.